In [2]:
import json
from pynput import keyboard
with open("./skills.json", "r") as f:
    skill_data = json.load(f)

skill_data

{'1': {'type': 'gcd',
  'name': '冰1',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '1',
  'consumption': 0},
 '2': {'type': 'gcd',
  'name': '冰2',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '!',
  'consumption': 1600},
 '3': {'name': '冰3',
  'type': 'gcd',
  'cast_time': 2.6,
  'recast_time': 2.5,
  'keyboard': 'q',
  'consumption': 2000},
 '4': {'name': '冰4',
  'type': 'gcd',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': 'V',
  'consumption': 0},
 '5': {'type': 'gcd',
  'name': '冰悖论',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '2',
  'consumption': 0},
 '6': {'type': 'gcd',
  'name': '火1',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '2',
  'consumption': 1600},
 '7': {'type': 'gcd',
  'name': '火2',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '@',
  'consumption': 1600},
 '8': {'type': 'gcd',
  'name': '火3',
  'cast_time': 2.6,
  'recast_time': 2.5,
  'keyboard': 'e',
  'consumption': 2000},
 '9': {'type': 'gc

In [1]:
import time
class BlackMage:
    def __init__(self):
        self.healthy = 60000
        self.magic = 10000
        self.skills = skill_data
        # gcd间隙插入的能力技数量，最多两个
        self.capability_count_gcdgap = 0
        # -3~3
        self.status = 0
        self.status_list = []
        self.ice_needle = 0
        self.paradox = "6"
        # 天语剩余时间
        self.enochian_remain_time = 0
    
    def input_keyboard(self):
        """监听一次按键，返回按键信息"""
        result = {'key': None, 'modifiers': []}
        
        def on_press(key):
            
            # 普通字符键 - 获取后立即停止监听
            if hasattr(key, 'char') and key.char:
                result['key'] = key.char
                return False  # 停止监听
        
        def on_release(key):
            
            # ESC 退出
            if key == keyboard.Key.esc:
                result['key'] = 'ESC'
                return False
        
        # 开始监听
        with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
            listener.join()
            key_info = result
            
            # 打印结果
            return key_info['key']
        
    def exam_magic(self, skill_id, skill_name):
        if self.status == 0:
            consum_magic = skill_data[skill_id]["consumption"]
            
        elif self.status > 0:
            if skill_name == "绝望" and self.magic < 800:
                return False
            if skill_id in ["6", "7", "8", "9", "10"]:
                if self.ice_needle > 0:
                    consum_magic = skill_data[skill_id]["consumption"] // 2
                else:
                    consum_magic = skill_data[skill_id]["consumption"]
            else:
                consum_magic = skill_data[skill_id]["consumption"]
            
        elif self.status < 0:
            if skill_id in ["1", "2", "3", "4", "5"]:
                consum_magic = 0
                if skill_id == "4":
                    self.ice_needle = 3
            else:
                consum_magic = skill_data[skill_id]["consumption"]
            

        if self.magic - consum_magic < 0:
            return False 
        
        self.magic -= consum_magic
        return True
        
        
        
    def exam_status(self, skill_id, skill_name):
        if self.status == 0:
            if "6" == skill_id:
                self.status += 1
            elif skill_id in ["7", "8"]:
                self.status = 3
            elif "1" == skill_id:
                self.status = -1
            elif skill_id in ["2", "3"]:
                self.status = -3
            
        elif self.status > 0:
            if skill_name == "绝望":
                self.status = 3
            elif skill_id in ["6", "10"]:
                self.status = 3 if self.status + 1 >=3 else self.status + 1
            elif skill_id in ["7", "8"]:
                self.status = 3
            elif "1" == skill_id:
                self.status = 0
            elif skill_id in ["2", "3"]:
                self.status = -3  

            
        elif self.status < 0:
            if "6" == skill_id:
                self.status = 0
            elif skill_id in ["7", "8"]:
                self.status = 3
            elif skill_id in ["1", "5"]:
                self.status = -3 if self.status -1 <=3 else self.status - 1
            elif skill_id in ["2", "3"]:
                self.status = -3



    def exam_paradox(self):
        if self.paradox != "6":
            return 
        
        if  self.status == 3 and len(self.status_list) > 1 and self.status_list[-1] == -3 and self.ice_needle == 3:
            self.paradox = "10"
            return
        elif self.status == -3 and len(self.status_list) > 1 and self.status_list[-1] == 3:
            self.paradox = "5"
            return
        else:
            self.paradox = "6"
            return

    def run_skill(self):
        user_input = self.input_keyboard()
        skill_id = [id for id, skill in self.skills.items() if skill["keyboard"] == user_input]
        if len(skill_id) > 1:
            
            skill_id = self.paradox

        else:
            skill_id = skill_id[0]

        
        skill_name = skill_data[skill_id]["name"]
        if self.exam_magic(skill_id, skill_name):
            print(f"casting... {skill_name}")
            print(f"remain magic: {self.magic}")
        else:
            print("remain magic not enough")
        self.exam_status(skill_id, skill_name)
        print(f"status: {self.status}, ice needle: {self.ice_needle}")

        print(f"{'-'*30}")
        self.exam_paradox()
        self.status_list.append(self.status)
        self.status_list = self.status_list[:5]
        pass
        
    
b = BlackMage()
for _ in range(7):
    b.run_skill()

NameError: name 'skill_data' is not defined

In [ ]:
[id for id, skill in skill_data.items() if skill["keyboard"] == "2"]

['5', '6', '10']

In [77]:
skill_data

{'1': {'type': 'gcd',
  'name': '冰1',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '1',
  'consumption': 0},
 '2': {'type': 'gcd',
  'name': '冰2',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '!',
  'consumption': 1600},
 '3': {'name': '冰3',
  'type': 'gcd',
  'cast_time': 2.6,
  'recast_time': 2.5,
  'keyboard': 'q',
  'consumption': 2000},
 '4': {'name': '冰4',
  'type': 'gcd',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': 'V',
  'consumption': 0},
 '5': {'type': 'gcd',
  'name': '冰悖论',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '2',
  'consumption': 0},
 '6': {'type': 'gcd',
  'name': '火1',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '2',
  'consumption': 1600},
 '7': {'type': 'gcd',
  'name': '火2',
  'cast_time': 2.44,
  'recast_time': 2.5,
  'keyboard': '@',
  'consumption': 1600},
 '8': {'type': 'gcd',
  'name': '火3',
  'cast_time': 2.6,
  'recast_time': 2.5,
  'keyboard': 'e',
  'consumption': 2000},
 '9': {'type': 'gc

In [1]:
import time
start = time.time()

In [2]:
start

1762627407.958776

In [5]:
time.time() - start

30.442523956298828